# Example: Neural Network Sampling

Before runnning this notebook, make sure that all neccesary libraries are installed:

In [11]:
using Flux
using Random
using Gogeta
using Gurobi
using JuMP
using QuasiMonteCarlo

In [12]:
dimension = 10

begin
    Random.seed!(1234);

    NN_model = Chain(
        Dense(dimension => 100, relu),
        Dense(100 => 1)
    )
end

Chain(
  Dense(10 => 100, relu),               # 1_100 parameters
  Dense(100 => 1),                      # 101 parameters
)                   # Total: 4 arrays, 1_201 parameters, 4.941 KiB.

In [3]:
# Set upper and lower input bounds
init_U = ones(dimension);
init_L = zeros(dimension);

In [4]:
# Formulate the MIP with heuristic bound tightening
jump_model = Model(Gurobi.Optimizer)
set_silent(jump_model)
NN_formulate!(jump_model, NN_model, init_U, init_L; bound_tightening="fast");

last_layer, _ = maximum(keys(jump_model[:x].data))
@objective(jump_model, Max, jump_model[:x][last_layer, 1])

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20


x[2,1]

In [8]:
samples = QuasiMonteCarlo.sample(1000, init_L, init_U, LatinHypercubeSample());


In [9]:
samples

10×1000 Matrix{Float64}:
 0.6525  0.5425  0.0745  0.7175  0.4435  …  0.4565  0.5695  0.3035  0.5145
 0.4605  0.1215  0.2465  0.5345  0.7415     0.4625  0.2435  0.4985  0.6285
 0.6265  0.8575  0.1935  0.9945  0.9185     0.4595  0.3895  0.9815  0.5195
 0.8625  0.9935  0.4855  0.7505  0.9725     0.7325  0.9695  0.1945  0.7815
 0.7595  0.3035  0.1715  0.7475  0.4775     0.7295  0.9595  0.2405  0.1745
 0.0215  0.6515  0.5135  0.8775  0.4475  …  0.6575  0.9495  0.3005  0.8175
 0.2845  0.7735  0.9725  0.1285  0.1095     0.7755  0.3715  0.2525  0.0205
 0.3735  0.8465  0.5635  0.6165  0.4795     0.2075  0.8245  0.8415  0.6585
 0.3575  0.1635  0.2315  0.3615  0.6785     0.1505  0.9405  0.3635  0.3125
 0.8405  0.7615  0.0805  0.6325  0.8485     0.4155  0.3685  0.6125  0.6595

In [10]:
@time x_opt, optimum = optimize_by_sampling!(jump_model, samples);
optimum

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

-0.0

In [6]:
NN_model(Float32.(x_opt))[] ≈ optimum

true

In [7]:
@time optimize!(jump_model)
objective_value(jump_model)

 14.102381 seconds (196.97 k allocations: 6.012 MiB, 0.02% compilation time)


0.07902304865851326